In [37]:
%pip install pmdarima

     ------------------------------------ 572.0/572.0 kB 642.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose

In [22]:
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [33]:
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )       


        # Get coordinates
        R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        #Visibility Extraction
        R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
        

        L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        #Visibility Extraction
        L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
               
        if R_V > L_V:
            angle = calculate_angle(R_hip, R_knee, R_ankle)
        else:
            angle = calculate_angle(L_hip, L_knee, L_ankle)
        counter+=1

                    # Visualize angle
        cv2.putText(image, str(angle), 
                        #tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                        tuple(np.multiply(L_knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA

                            )
        if counter == 4:
            counter = 0
            iter += 1
            
            c = 0
            avg_y = 0
            for i in range(11):
                if landmarks[i+11].visibility > 0.9:
                    avg_y += landmarks[i+11].y
                    c += 1
            avg_y / c
            

            avg_y = (landmarks[11].y + landmarks[13].y + landmarks[15].y)/3

            if angle > 160:
                data["spike/deep"].append(-1)
                data["values"].append(avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            elif angle < 110:
                data["spike/deep"].append(1)
                data["values"].append(avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            else:
                data["spike/deep"].append(0)
                data["values"].append(avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                
                
        
        cv2.imshow('Mediapipe Feed', image)
        
        if (cv2.waitKey(10) & 0xFF == ord('q') | (iter == 100)):
            break

    cap.release()
    cv2.destroyAllWindows()

In [101]:
df.to_csv("Squats_filtered.csv" , index = False)

In [34]:
#To Dataframe
df_raw = pd.DataFrame(data = data)

df_raw.to_csv("Squats.csv" , index = False)
df_raw = df_raw.drop(index=range(28))

#without Outliers
df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [35]:
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')
df_raw.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [61]:
res = seasonal_decompose(df["values"],model = "additive",period=7)
res.plot();

In [72]:
auto_arima(df["values"], m=7, trace=True, seasonal=True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[7] intercept   : AIC=inf, Time=0.59 sec
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=-93.078, Time=0.04 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=-134.395, Time=0.34 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=-139.917, Time=0.13 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=112.607, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=-135.222, Time=0.04 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=-138.418, Time=0.33 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=-138.366, Time=0.23 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=-102.209, Time=0.08 sec
 ARIMA(1,0,1)(0,0,1)[7] intercept   : AIC=-141.701, Time=0.22 sec
 ARIMA(1,0,1)(0,0,0)[7] intercept   : AIC=-140.067, Time=0.12 sec
 ARIMA(1,0,1)(1,0,1)[7] intercept   : AIC=inf, Time=0.43 sec
 ARIMA(1,0,1)(0,0,2)[7] intercept   : AIC=-139.857, Tim

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  114
Model:               SARIMAX(2, 0, 1)   Log Likelihood                  82.289
Date:                Sun, 05 Mar 2023   AIC                           -154.578
Time:                        19:41:48   BIC                           -140.897
Sample:                             0   HQIC                          -149.025
                                - 114                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1207      0.032      3.829      0.000       0.059       0.183
ar.L1          1.3417      0.159      8.424      0.000       1.030       1.654
ar.L2         -0.6814      0.166     -4.114      0.000      -1.006      -0.357
ma.L1         -0.7504      0.135     -5.548      0.000      -1.016      -0.485
sigma2         0.0137      0.003      4.712      0.000       0.008       0.019
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.22
Prob(Q):                              0.98   Prob(JB):                         0.54
Heteroskedasticity (H):               0.58   Skew:                            -0.17
Prob(H) (two-sided):                  0.10   Kurtosis:                         2.62
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [83]:
train_df = df.iloc[:80]
test_df = df.iloc[80:]
model = SARIMAX(train_df["values"], order=(2, 0, 1))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = test_df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)


c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No suppor

<AxesSubplot:>

In [84]:
from statsmodels.tools.eval_measures import rmse
accuracy_no_deeps = 1- rmse(test_df["values"], prediction)
print(accuracy_no_deeps)

0.6646943137267131


In [73]:
auto_arima(df["values"], exogenous=df["spike/deep"], seasonal=True , m = 7, trace  =True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[7] intercept   : AIC=inf, Time=0.63 sec
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=-93.078, Time=0.03 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=-134.395, Time=0.33 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=-139.917, Time=0.14 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=112.607, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=-135.222, Time=0.04 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=-138.418, Time=0.26 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=-138.366, Time=0.23 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=inf, Time=0.54 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=-102.209, Time=0.08 sec
 ARIMA(1,0,1)(0,0,1)[7] intercept   : AIC=-141.701, Time=0.22 sec
 ARIMA(1,0,1)(0,0,0)[7] intercept   : AIC=-140.067, Time=0.17 sec
 ARIMA(1,0,1)(1,0,1)[7] intercept   : AIC=inf, Time=0.56 sec
 ARIMA(1,0,1)(0,0,2)[7] intercept   : AIC=-139.857, Tim

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  114
Model:               SARIMAX(2, 0, 1)   Log Likelihood                  82.289
Date:                Sun, 05 Mar 2023   AIC                           -154.578
Time:                        19:42:41   BIC                           -140.897
Sample:                             0   HQIC                          -149.025
                                - 114                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1207      0.032      3.829      0.000       0.059       0.183
ar.L1          1.3417      0.159      8.424      0.000       1.030       1.654
ar.L2         -0.6814      0.166     -4.114      0.000      -1.006      -0.357
ma.L1         -0.7504      0.135     -5.548      0.000      -1.016      -0.485
sigma2         0.0137      0.003      4.712      0.000       0.008       0.019
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.22
Prob(Q):                              0.98   Prob(JB):                         0.54
Heteroskedasticity (H):               0.58   Skew:                            -0.17
Prob(H) (two-sided):                  0.10   Kurtosis:                         2.62
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [98]:
train_df = df.iloc[:80]
test_df = df.iloc[80:]
model = SARIMAX(endog=train_df["values"], order=(2, 0, 1), exog=train_df["spike/deep"])
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end,exog = test_df["spike/deep"]).rename("prediction")
ax = test_df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index

<AxesSubplot:>

In [100]:
prediction

80     0.559652
81     0.420653
82     0.276963
83     0.278831
84     0.276496
85     0.278139
86     0.558251
87     0.559694
88     0.557763
89     0.559026
90     0.557265
91     0.276133
92     0.274526
93     0.275484
94     0.274014
95     0.274842
96     0.555730
97     0.556442
98     0.555206
99     0.555815
100    0.554678
101    0.555194
102    0.271912
103    0.272344
104    0.271375
105    0.271733
106    0.270836
107    0.553361
108    0.552528
109    0.552759
110    0.551983
111    0.269927
112    0.551436
113    0.551567
Name: prediction, dtype: float64

In [99]:
accuracy = 1- rmse(test_df["values"], prediction)
print(accuracy)

0.9173200783498131


In [ ]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)

# fit SARIMA model to time series
model = sm.tsa.statespace.SARIMAX(df["values"], order=order, seasonal_order=seasonal_order)
results = model.fit()

# make predictions for next 30 seconds (assuming last observation was at 15:20:30)
start_time = df.index[-1] + 1
end_time = start_time + 30
forecast = results.predict(start=df.index[0], end=df.index[-1])
df["forecast"] = forecast
print(forecast)